# Analyze past Allegheny County primaries

In [1]:
import pandas as pd
from PyPDF2 import PdfReader
import re

In [2]:
### Calculate relevant statistics, append year suffix to columns
def calculate_stats(df, year):
    df = df.groupby('precinct').agg({
        'd_reg': 'sum',
        'd_cast': 'sum'
    }).reset_index()

    df['turnout'] = df.d_cast / df.d_reg
    df['ballots_pct_of_total'] = df.d_cast / df.d_cast.sum()
    df['reg_pct_of_total'] = df.d_reg / df.d_reg.sum()

    df.columns = [f'{i}_{year}' if i not in ['precinct'] else i for i in df.columns]
    df = df.set_index('precinct')
    return df

### Remove ward and district and ward markings, but leave ward for Pittsburgh
def format_precinct_name(precinct):
    precinct = precinct.replace(' WD', ' WARD')
    precinct = precinct.replace(' WRD', ' WARD')
    precinct = re.sub(r' DIST.*', '', precinct)

    if precinct.startswith('PITTSBURGH'):
        return precinct
    else:
        return re.sub(r' WARD.*', '', precinct)

In [3]:
def _parse_pdf_precinct_stats(index, precincts, precinct, text, d_reg_regex, d_cast_regex):
    precinct_text_index = text.index(precinct)

    if index == len(precincts) - 1:
        next_precinct_text_index = len(text)
    else:
        next_precinct_text_index = text.index(precincts[index + 1])

    precinct_text = text[precinct_text_index:next_precinct_text_index]
    d_reg = re.findall(d_reg_regex, precinct_text)[0][0]
    d_cast = re.findall(d_cast_regex, precinct_text)[0][0]

    return { 'precinct': precinct, 'd_reg': d_reg, 'd_cast': d_cast }

REGEX_PRECINCT = "STATISTICS\n(.*)\n                                                      VOTES  PERCENT"
REGEX_D_CAST_15 = "BALLOTS CAST - DEMOCRATIC .  .  .  .  .  .\s+(\d+)\s+(.*)(\n)?\s+BALLOTS CAST - REPUBLICAN"
REGEX_D_REG_15 = "REGISTERED VOTERS - DEMOCRATIC  .  .  .  .\s+(\d+)   (.*)(\n)?           REGISTERED VOTERS - REPUBLICAN"

def parse_pdf_results(filename):
    reader = PdfReader(filename)
    text = ""

    for page in reader.pages:
        text += page.extract_text() + "\n"

    text = re.sub("Page \d+\n", "", text)
    precincts = re.findall(REGEX_PRECINCT, text)

    d_cast_regex = REGEX_D_CAST_15
    d_reg_regex = REGEX_D_REG_15

    precincts_with_figures = [
        _parse_pdf_precinct_stats(index, precincts, precinct, text, d_reg_regex, d_cast_regex)
        for index, precinct in enumerate(precincts)
    ]

    df = pd.DataFrame.from_dict(precincts_with_figures)
    df['precinct'] = df.precinct.str.replace(r'^\d+ ', '', regex=True)
    df['d_cast'] = df.d_cast.astype(int)
    df['d_reg'] = df.d_reg.astype(int)

    return df

### Import voting data

##### Import P13

##### Import P15

In [4]:
p15 = parse_pdf_results('input/primary_2015.pdf')

p15['precinct'] = p15.precinct.apply(format_precinct_name)
p15 = calculate_stats(p15, '15')
p15.head(1)

,d_reg_15,d_cast_15,turnout_15,ballots_pct_of_total_15,reg_pct_of_total_15
precinct,,,,,
ALEPPO,472,74,0.15678,0.00065,0.00093


##### Import P17

In [5]:
p17 = pd.read_csv('input/primary_2017.csv')
p17 = p17[['Precinct Name', 'Democratic Reg Voters', 'Democratic Ballots Cast']]
p17.columns = ['precinct', 'd_reg', 'd_cast']

p17['precinct'] = p17.precinct.apply(format_precinct_name)
p17 = calculate_stats(p17, '17')
p17.head(1)

,d_reg_17,d_cast_17,turnout_17,ballots_pct_of_total_17,reg_pct_of_total_17
precinct,,,,,
ALEPPO,520,123,0.236538,0.001016,0.000967


##### Import P19

In [6]:
p19 = pd.read_csv('input/primary_2019.csv')
p19 = p19[['Precinct Name', 'Democratic Reg Voters', 'Democratic Ballots Cast']]
p19.columns = ['precinct', 'd_reg', 'd_cast']

p19['d_cast'] = p19.d_cast.str.replace(r'.\s', '', regex=True).astype(int)

p19['precinct'] = p19.precinct.apply(format_precinct_name)
p19 = calculate_stats(p19, '19')
p19.head(1)

,d_reg_19,d_cast_19,turnout_19,ballots_pct_of_total_19,reg_pct_of_total_19
precinct,,,,,
ALEPPO,527,98,0.185958,0.00083,0.000955


### Merge primaries; adjust precinct names

### Analyze primaries

##### PGH vs. non-PGH